Christopher Wilbar   
MSDS_460-DL_SEC59   
Homework #3

In [1]:
#Import needed packages
import numpy as np
import pandas as pd
import itertools
from gurobipy import *

# Problem 3

I have modified the example tsp code found on gurobi examples here:   
https://www.gurobi.com/documentation/8.1/examples/tsp_py.html

#Copyright 2019, Gurobi Optimization, LLC

The strucutre is the same. The code has been modified to reflect a non-symmetric tsp problem, requiring the following changes:   
1. subtourelim function modified to create new lazy constratints based on permutations instead of combinations
2. Created dictonary of fixed "distances" which here refelct surgery times instead of generating random distances between points
3. constraints modified to sum each row and each column to 1 instead of edges for each node to 2
4. added constraint to insure that each diagonal (i,i) is 0

In [2]:
# Callback - use lazy constraints to eliminate sub-tours

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # make a list of edges selected in the solution
        vals = model.cbGetSolution(model._vars)
        selected = tuplelist((i,j) for i,j in model._vars.keys() if vals[i,j] > 0.5)
        # find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n:
            # add subtour elimination constraint for every pair of cities in tour
            model.cbLazy(quicksum(model._vars[i,j]
                                  for i,j in itertools.permutations(tour, 2)) <= len(tour)-1)

In [3]:
# Given a tuplelist of edges, find the shortest subtour

def subtour(edges):
    unvisited = list(range(n))
    cycle = range(n+1) # initial length has 1 more city
    while unvisited: # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i,j in edges.select(current,'*') if j in unvisited]
        if len(cycle) > len(thiscycle):
            cycle = thiscycle
    return cycle

## 3.1 | 5 Patients

In [4]:
#Define dictonary for surgery length times
sijvalues = [[0,20,15,8,6],[15,0,18,9,28],[24,23,0,13,13],[15,27,8,0,14],[8,17,24,15,0]]

sijvaluesdf = pd.DataFrame(sijvalues)

n = sijvaluesdf.shape[1]

sij = {(i,j) : sijvaluesdf.iloc[i,j]
        for i in range(n) for j in range(n)}

In [5]:
#Initialize model
m = Model("surgery")

Academic license - for non-commercial use only


In [6]:
#Create Variables. Define variables as binary
vars = m.addVars(sij.keys(), obj=sij, vtype=GRB.BINARY, name='x')

In [7]:
#Define Constraints
#For every surgery j only one unique surgery i can immediately proceed j.
m.addConstrs(vars.sum(i,'*') == 1 for i in range(n))

#For every surgery i only one unique surgery j can immediately follow i.
m.addConstrs(vars.sum('*',j) == 1 for j in range(n))

#Exclude the self-loop decision variables (xii) by fixing the diagonal elements of X to equal zero.
m.addConstrs(vars.sum(i,i) == 0 for i in range(n))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>}

In [8]:
# Optimize model
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

vals = m.getAttr('x', vars)
selected = tuplelist((i,j) for i,j in vals.keys() if vals[i,j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 15 rows, 25 columns and 55 nonzeros
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 91.0000000
Presolve removed 5 rows and 5 columns
Presolve time: 0.00s
Presolved: 10 rows, 20 columns, 40 nonzeros
Variable types: 0 continuous, 20 integer (20 binary)

Root relaxation: objective 5.400000e+01, 8 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   57.50000    0    8   91.00000   57.50000  36.8%     -    0s
H    0     0                      61.0000000   57.50000  5.74%     -    0s
*    0     0               0      58.0000000   58.00000  0.00%     -

In [9]:
# Store results for 5 patient version and output
#"Rename by adding One to get to the list in problem that starts at 1"
tour5 = [x+1 for x in tour]
objVal5 = m.objVal

print('')
print('Optimal Schedule: %s' % str(tour5))
print('Minimal Time: %g' % objVal5, 'minutes')
print('')


Optimal Schedule: [1, 2, 4, 3, 5]
Minimal Time: 58 minutes



## 3.2 | 10 Patients 

In [10]:
#Define dictonary for surgery length times
sijvalues = [
[0,9,12,26,11,24,12,13,17,15],
[24,0,28,23,22,5,7,18,9,23],
[19,30,0,30,15,22,25,15,28,15],
[18,10,27,0,28,12,16,19,22,7],
[5,16,11,7,0,25,27,30,23,15],
[7,26,6,17,6,0,28,20,13,28],
[23,26,20,20,24,30,0,16,18,27],
[23,20,22,8,18,10,14,0,14,12],
[7,13,9,19,29,27,18,23,0,30],
[16,10,11,11,28,26,6,11,12,0]
]

sijvaluesdf = pd.DataFrame(sijvalues)

n = sijvaluesdf.shape[1]

sij = {(i,j) : sijvaluesdf.iloc[i,j]
        for i in range(n) for j in range(n)}

In [11]:
#Initialize model
m = Model("surgery")

In [12]:
#Create Variables. Define variables as binary
vars = m.addVars(sij.keys(), obj=sij, vtype=GRB.BINARY, name='x')

In [13]:
#Define Constraints
#For every surgery j only one unique surgery i can immediately proceed j.
m.addConstrs(vars.sum(i,'*') == 1 for i in range(n))

#For every surgery i only one unique surgery j can immediately follow i.
m.addConstrs(vars.sum('*',j) == 1 for j in range(n))

#Exclude the self-loop decision variables (xii) by fixing the diagonal elements of X to equal zero.
m.addConstrs(vars.sum(i,i) == 0 for i in range(n))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>}

In [14]:
# Optimize model
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

vals = m.getAttr('x', vars)
selected = tuplelist((i,j) for i,j in vals.keys() if vals[i,j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 30 rows, 100 columns and 210 nonzeros
Variable types: 0 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolved: 20 rows, 90 columns, 180 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)

Root relaxation: objective 8.800000e+01, 15 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   89.33333    0   14          -   89.33333      -     -    0s
H    0     0                      95.0000000   89.33333  5.96%     -    0s
H    0     0                      92.0000000   89.33333  2.90%     -    0s
     0     0     cutoff    0    

In [15]:
# Store results for 5 patient version and output
#"Rename by adding One to get to the list in problem that starts at 1"
tour10 = [x+1 for x in tour]
objVal10 = m.objVal

print('')
print('Optimal Schedule: %s' % str(tour10))
print('Minimal Time: %g' % objVal10, 'minutes')
print('')


Optimal Schedule: [1, 2, 6, 5, 3, 8, 4, 10, 7, 9]
Minimal Time: 92 minutes



## 3.3 | 15 Patients 

In [16]:
#Define dictonary for surgery length times
sijvalues = [
[0,15,27,7,19,7,22,26,29,6,5,18,30,11,25],
[24,0,9,23,21,15,25,8,8,9,16,26,22,8,16],
[22,5,0,28,14,22,30,7,22,26,8,5,25,22,14],
[6,20,13,0,23,17,5,17,11,15,30,30,26,21,26],
[27,18,19,17,0,29,18,26,12,22,16,17,14,24,10],
[8,29,21,8,14,0,6,27,25,14,22,13,14,7,9],
[24,24,5,25,16,27,0,24,8,24,12,15,26,29,28],
[16,23,10,13,6,9,15,0,14,12,9,21,14,17,23],
[5,23,26,18,21,12,28,29,0,8,6,27,23,11,10],
[18,18,17,7,7,8,12,13,28,0,20,5,27,25,29],
[10,18,28,26,24,15,17,21,25,25,0,30,16,15,20],
[28,15,17,6,28,30,17,23,14,22,6,0,27,19,28],
[10,19,27,25,17,28,18,25,8,25,13,17,0,7,28],
[12,10,25,26,22,7,20,18,16,25,19,24,19,0,16],
[11,28,13,22,10,21,23,24,19,23,19,16,19,19,0]
]

sijvaluesdf = pd.DataFrame(sijvalues)

n = sijvaluesdf.shape[1]

sij = {(i,j) : sijvaluesdf.iloc[i,j]
        for i in range(n) for j in range(n)}

In [17]:
#Initialize model
m = Model("surgery")

In [18]:
#Create Variables. Define variables as binary
vars = m.addVars(sij.keys(), obj=sij, vtype=GRB.BINARY, name='x')

In [19]:
#Define Constraints
#For every surgery j only one unique surgery i can immediately proceed j.
m.addConstrs(vars.sum(i,'*') == 1 for i in range(n))

#For every surgery i only one unique surgery j can immediately follow i.
m.addConstrs(vars.sum('*',j) == 1 for j in range(n))

#Exclude the self-loop decision variables (xii) by fixing the diagonal elements of X to equal zero.
m.addConstrs(vars.sum(i,i) == 0 for i in range(n))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>}

In [20]:
# Optimize model
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

vals = m.getAttr('x', vars)
selected = tuplelist((i,j) for i,j in vals.keys() if vals[i,j] > 0.5)

tour = subtour(selected)
assert len(tour) == n

Changed value of parameter lazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 45 rows, 225 columns and 465 nonzeros
Variable types: 0 continuous, 225 integer (225 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 15 rows and 15 columns
Presolve time: 0.00s
Presolved: 30 rows, 210 columns, 420 nonzeros
Variable types: 0 continuous, 210 integer (210 binary)

Root relaxation: objective 1.110000e+02, 23 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     114.0000000  114.00000  0.00%     -    0s

Cutting planes:
  Lazy constraints: 3

Explored 0 nodes (28 simplex iterations) in 0.05 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 114 

Optimal so

In [21]:
# Store results for 5 patient version and output
#"Rename" by adding One to get to the list in problem that starts at 1
tour15 = [x+1 for x in tour]
objVal15 = m.objVal

print('')
print('Optimal Schedule: %s' % str(tour15))
print('Minimal Time: %g' % objVal15, 'minutes')
print('')


Optimal Schedule: [1, 10, 12, 4, 7, 3, 2, 8, 5, 9, 11, 13, 14, 6, 15]
Minimal Time: 114 minutes



# Extra Credit

In [ ]:
# I know there must be a more elegant way but was struggling so decided to just type everything out explicitly

In [22]:
#Demand for each type of board (5,4,3)
Demand = [60,60,90]

In [23]:
#Matrix for Cuts per arrangment
Cuts5 = [2,1,1,0,0,0]
Cuts4 = [0,1,0,2,1,0]
Cuts3 = [0,0,2,0,2,3]

In [24]:
# Initialize Model
m = Model("Boards")

In [25]:
#Define Variables
x1 = m.addVar(lb=0, vtype = 'I', name = "arrangement1")
x2 = m.addVar(lb=0, vtype = 'I', name = "arrangement2")
x3 = m.addVar(lb=0, vtype = 'I', name = "arrangement3")
x4 = m.addVar(lb=0, vtype = 'I', name = "arrangement4")
x5 = m.addVar(lb=0, vtype = 'I', name = "arrangement5")
x6 = m.addVar(lb=0, vtype = 'I', name = "arrangement6")

In [26]:
# Define Objective Function
m.setObjective(x1+x2+x3+x4+x5+x6, GRB.MINIMIZE)

In [27]:
#Add Constraints
m.addConstr(Cuts5[0]*x1 + Cuts5[1]*x2+ Cuts5[2]*x3+ Cuts5[3]*x4+ Cuts5[4]*x5+ Cuts5[5]*x6 >= Demand[0])
m.addConstr(Cuts4[0]*x1 + Cuts4[1]*x2+ Cuts4[2]*x3+ Cuts4[3]*x4+ Cuts4[4]*x5+ Cuts4[5]*x6 >= Demand[1])
m.addConstr(Cuts3[0]*x1 + Cuts3[1]*x2+ Cuts3[2]*x3+ Cuts3[3]*x4+ Cuts3[4]*x5+ Cuts3[5]*x6 >= Demand[2])

<gurobi.Constr *Awaiting Model Update*>

In [28]:
#Solve
m.optimize()

Optimize a model with 3 rows, 6 columns and 9 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 9e+01]
Found heuristic solution: objective 120.0000000
Presolve time: 0.00s
Presolved: 3 rows, 6 columns, 9 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 8.250000e+01, 4 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   82.50000    0    1  120.00000   82.50000  31.3%     -    0s
H    0     0                      83.0000000   82.50000  0.60%     -    0s

Explored 1 nodes (4 simplex iterations) in 0.04 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 83 120 

Optimal solution found (tolerance 1.00e-04)
Best objective 8.300000000

In [29]:
#Print Results
for v in m.getVars():
    print('%s %g' % (v.varName, v.x))

print('Minimum Boards: %g' % m.objVal)

arrangement1 30
arrangement2 -0
arrangement3 0
arrangement4 8
arrangement5 45
arrangement6 -0
Minimum Boards: 83
